## Research on the Hacker News posts

### Dataset

This research will be run on [the Hacker News Posts dataset](https://www.kaggle.com/hacker-news/hacker-news-posts).

The dataset contains ~300000 rows and following columns:
- `id`
- `title`
- `url`
- `num_points`
- `num_comments`
- `author`
- `created_at`

### Goal

In this research I'm going to answer the following questions:
1. Do `Ask HN` or `Show HN` receive more comments on average?
2. Do posts created at a certain time receive more comments on average?

### Cleaning

In [1]:
from csv import reader
opened_file = open('hacker_news.csv')
read_file = reader(opened_file)
hn = list(read_file) # Create the list of lists
opened_file.close()

hn[:5]

[['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at'],
 ['12224879',
  'Interactive Dynamic Video',
  'http://www.interactivedynamicvideo.com/',
  '386',
  '52',
  'ne0phyte',
  '8/4/2016 11:52'],
 ['10975351',
  'How to Use Open Source and Shut the Fuck Up at the Same Time',
  'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/',
  '39',
  '10',
  'josep2',
  '1/26/2016 19:30'],
 ['11964716',
  "Florida DJs May Face Felony for April Fools' Water Joke",
  'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/',
  '2',
  '1',
  'vezycash',
  '6/23/2016 22:20'],
 ['11919867',
  'Technology ventures: From Idea to Enterprise',
  'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429',
  '3',
  '1',
  'hswarna',
  '6/17/2016 0:01']]

Deal with headers:

In [2]:
headers = hn[0]
hn = hn[1:]

print(headers)
for row in hn[:5]:
    print(row)

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']
['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52']
['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30']
['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20']
['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']
['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']


### Analysis

Separate posts:

In [3]:
ask_posts = []
show_posts = []
other_posts = []

for row in hn:
    title = row[1]
    if title.lower().startswith('ask hn'):
        ask_posts.append(row)
    elif title.lower().startswith('show hn'):
        show_posts.append(row)
    else:
        other_posts.append(row)
        
print("Number of the 'Ask HN' posts: ", len(ask_posts))
print("Number of the 'Show HN' posts: ", len(show_posts))
print("Number of the other posts: ", len(other_posts))

Number of the 'Ask HN' posts:  1744
Number of the 'Show HN' posts:  1162
Number of the other posts:  17194


Let's find the total number of comments for each type of posts:

In [4]:
def column_avg(a_list, index):
    total = 0
    for row in a_list:
        value = row[index]
        total += int(value)
    avg = total / len(a_list)
    return avg

avg_ask_comments = column_avg(ask_posts, 4)
print("Average number of comments on 'Ask HN' posts: {:g}"\
      .format(avg_ask_comments))

avg_show_comments = column_avg(show_posts, 4)
print("Average number of comments on 'Show HN' posts: {:g}"\
      .format(avg_show_comments))

Average number of comments on 'Ask HN' posts: 14.0384
Average number of comments on 'Show HN' posts: 10.3167


The finding is that `Ask HN` posts receive ~40% more comments than `Show HN` ones.

The next step of the research is to determine the time of the day when most of the comments left.

In [5]:
import datetime as dt

# Create a dataset with the time of the post creation and the number of its comments
result_list = []

for row in ask_posts:
    created_at = row[6]
    num_comments = row[4]
    result_row = [created_at, num_comments]
    result_list.append(result_row)
    
counts_by_hour = {}
comments_by_hour = {}

for row in result_list:
    # Parse 'created_at' column and find the hour of creation
    created_at_str = row[0]
    created_at_dt = dt.datetime.strptime(created_at_str,\
                                         "%m/%d/%Y %H:%M")
    created_at_hour = created_at_dt.strftime('%H')
    
    num_comments = int(row[1])
    '''
    Find the number of posts and 
    the number of comments per hour
    '''
    if created_at_hour not in counts_by_hour:
        counts_by_hour[created_at_hour] = 1
        comments_by_hour[created_at_hour] = num_comments
    else:
        counts_by_hour[created_at_hour] += 1
        comments_by_hour[created_at_hour] += num_comments

'''
Find the average number of comments per post for posts created
during each hour
'''
avg_by_hour = []

for hour in counts_by_hour:
    avg = comments_by_hour[hour] / counts_by_hour[hour]
    avg_row = [hour, avg]
    avg_by_hour.append(avg_row)
 
# Swap columns in avg_by_hour
swap_avg_by_hour = []

for row in avg_by_hour:
    swap_avg_by_hour.append([row[1], row[0]])
    
sorted_swap = sorted(swap_avg_by_hour, reverse=True)

print("Top 5 Hours for Ask Posts Comments")

for row in sorted_swap[:5]:
    hour = dt.datetime.strptime(row[1], '%H')
    hour = dt.datetime.strftime(hour, '%H:%M')
    avg = row[0]
    print("{}: {:.2f} average comments per post"\
          .format(hour, avg))

Top 5 Hours for Ask Posts Comments
15:00: 38.59 average comments per post
02:00: 23.81 average comments per post
20:00: 21.52 average comments per post
16:00: 16.80 average comments per post
21:00: 16.01 average comments per post


### Conclusion

The key finding is that posts `Ask HN` receive the most number of comments per post at 15:00 *(Eastern Time in the US)*, thus it is 23:00 (Eastern European Summer Time in Cyprus)